# La théorie des jeux

Auteur : Philippe Mathieu, [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr), email : philippe.mathieu@univ-lille.fr

Contributeurs : Louisa Fodil (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Creation : 18/01/2018


## Introduction

La théorie des jeux est la discipline qui étudie de manière formelle les interactions que peuvent avoir plusieurs individus dans une situation conflictuelle. Le terme "jeu" n'a a priori rien de ludique, Von Neumann l'ayant par exemple développée pour la modélisation des relations Franco-Américaines de 1945 et notamment lors de l'épisode de la baie des cochons. La théorie des jeux décompose les jeux en fonction du nombre de joueurs (généralement 2), puis principalement en deux grandes familles : les jeux simultanés, dans lesquels les joueurs jouent simultanément et donc ne savent pas en jouant ce que joue l'adversaire (papier-feuiile-ciseaux par exemple) et les jeux non simultanés où chacun joue tour à tour (les échecs par exemple). 

Nous nous plaçons ici dans le cadre des **jeux à deux joueurs, simultanés**. On représente en général ce type de jeu par une matrice de gains. Les coups d'un des deux joueurs se trouvent en abscisse et ceux de l'autre en ordonnée. L'intersection de chaque case caractérise une situation de jeu. On y note les points qui seront distribués à chacun des deux joueurs. Pour information, la théorie des jeux décompose encore ces jeux en deux autres familles, les jeux à somme nulle dans lesquels tout ce que perd l'un est gagné par l'autre, et les jeux à somme non nulle dans lesquels les points distribués sont différents selon les coups joués.

Cette feuille a pour objectif de montrer comment représenter et générer des jeux, puis comment calculer les différents équilibres d'un jeu.

## Une matrice de gains

Tout d'abord, il est necessaire de pouvoir coder une matrice de gains. Une classe `Game` va nous
permettre de stocker les gains des deux joueurs pour une situation donnée. 
Un objet Game prend en paramètre un tableau de couples d'entiers correspondant aux scores de chaque issue du jeu, ainsi qu'un tableau des noms des actions correspondantes.
En interne, Le package *numpy* offre un objet `Array` idéal pour stocker et manipuler cela. Pour un jeu à `n` coups, nous créons un Array `n*n` de couples de valeurs `(x,y)` avec `x` le gain du joueur 1 et `y` le gain du joueur 2.

In [ ]:
import numpy as np
import math

class Game:
    def __init__(self, tab, actions):
        self.actions=actions
        m=np.array(tab,dtype=[('x', object), ('y', object)])
        self.size = int(math.sqrt(len(tab)))
        self.scores=m.reshape(self.size,self.size)

In [ ]:
# Quelques jeux célèbres

# guerre des sexes
gs=[(3,2),(1,1),(0,0),(2,3)]     

# chicken game
gs=[(3,2),(1,1),(0,0),(2,3)]     

mp=[(1,-1),(-1,1),(-1,1),(1,-1)] 

# papier-feuille-ciseaux
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)] 

# Dilemme du prisonnier
dp =[(3,3),(0,5),(5,0),(1,1)]
g = Game(dp,['C','D'])
g.scores

# La notion d'équilibre

Les économistes se sont très vite emparés de ces modèles pour étudier différents problèmes économiques. Afin d'identifier la meilleure manière de jouer à un jeu fixé, ils se sont basés sur la notion abstraite de rationalité du joueur. Partant du principe que, dans le pire des cas, l'adversaire est aussi rationnel, le raisonnement que je m'applique doit donc aussi être appliqué à mon adversaire. Très vite, plusieurs définitions de la rationalité sont apparues, définissant ainsi un (ou éventuellement plusieurs) point fixe d'un jeu, point vers lequel rationnellement, si tout le monde réfléchis de manière identique, on parviendra :
1. *L'équilibre de Nash*, du nom de son inventeur, [John Forbes Nash](https://fr.wikipedia.org/wiki/John_Forbes_Nash). L'équilibre de Nash est une situation de non regret de chacun des joueurs. *"nous avons joué simultanément, mais maintenant que je sais ce que l'autre a joué, je n'ai aucun regret"*
2. *L'optimum de Pareto*, du nom de son inventeur, [Vilfredo Pareto](https://fr.wikipedia.org/wiki/Vilfredo_Pareto). l'optimum de Pareto est une situation où aucune situation n'est supérieure pour les deux joueurs, et donc ils n'ont aucun interêt collectivement à changer.
3. *L'équilibre en stratégies dominantes*. Un joueur "rationnel" ne jouera jamais une de ses stratégies si elle dominée par une autre de ses stratégies. On peut donc "simplifier" un jeu en éliminant itérativement les stratégies dominées.



## L'équilibre de Nash

Pour calculer la situation de "non regret" correspondant à l'équilibre de Nash il suffit de noter les réponses du joueur 1 les plus adaptées à chaque stratégie du joueur 2 (donc calculer les max(x) dans la chaque colonne), puis calculer les meilleures réponses du joueur 2 aux stratégies du joueur 1 (donc calculer les max(y) dans chaque ligne). Si une issue possède 2 max, c'est un équilibre de Nash. Selon les jeux, il peut bien sûr y en avoir 1, plusieurs ou pas du tout. 
L'usage d'un `np.Array` facilite énormément les choses puisqu'il est possible d'avoir les vecteurs de valeurs max en ligne ou en colonne. Il suffit donc de fabriquer des matrices booléennes dans chaque cas et d'en faire un `ou` logique.

In [ ]:
def getNash(self):
    max_x = np.matrix(self.scores['x'].max(0)).repeat(self.size, axis=0)
    bool_x = self.scores['x'] == max_x
    max_y = np.matrix(self.scores['y'].max(1)).transpose().repeat(self.size, axis=1)
    bool_y = self.scores['y'] == max_y
    bool_x_y = bool_x & bool_y
    result = np.where(bool_x_y == True)
    listOfCoordinates= list(zip(result[0], result[1]))
    return listOfCoordinates

#### Application à la guerre des sexes 
Dans [la guerre des sexes](https://en.wikipedia.org/wiki/Battle_of_the_sexes_(game_theory)) un couple s'est donné rendez-vous pour la soirée, mais aucun ne parvient à se souvenir si c'est pour assister à un match de foot ou aller à l'opéra. 
Le mari préférerait aller voir le foot, la femme aimerait aller à l'opéra. 
Tous deux préfèrent cependant aller au même endroit plutôt que d'être seuls.
Si l'homme va au stade et que sa femme s'y trouve, il obtient 3.
Si il est seul au stade, il obtient 1.
Si il va à l'opéra et que sa femme est au stade, il obtient 0.
Si il va à l'opéra et que sa femme est aussi à l'opéra, il obtient 2.
(Même chose pour la femme en inversant opéra et stade)
On obtient donc la matrice gs suivante sur laquelle on peut tester l'équilibre de Nash.

In [ ]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stade'])

# On récupére les indices du/des équilibres de Nash
listOfCoordinates = getNash(g)
print("Les coordonnées des équilibres de Nash : ")
print(listOfCoordinates)

# On imprime les coups correspondants à ces équilibres
print("Les coups correspondants : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# On imprimes les scores correspondants
print("Les scores correspondants : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

On remarque ici que le jeu possède deux équilibres de Nash qui correspondent aux situations où le couple est ensemble.

In [ ]:
# Exercice : Trouvez le/les équilibres de Nash (s'ils existent) pour le jeu Pierre Feuille Ciseaux 

rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

## L'optimum de Pareto



In [ ]:
def isPareto(self, t, s):
    return True if (len(s)==0) else (s[0][0]<=t[0] or s[0][1]<=t[1]) and isPareto(self,t, s[1:])

def getPareto(self):
        x = 0
        y = 0
        res = list()
        liste = self.scores.flatten()
        for s in liste:
            if (x == self.size):
                x = 0
                y = y + 1
            if(isPareto(self,s,liste)):res.append((x , y))
            x = x + 1
        return res

#### Reprenons le jeu de la guerre des sexes

In [ ]:
gs=[(3,2),(1,1),(0,0),(2,3)]
g = Game(gs,['Opera','Stade'])

# On récupére les indices du/des équilibres de Pareto
listOfCoordinates = getPareto(g)
print("Les coordonnées des équilibres de Pareto : ")
print(listOfCoordinates)

# On imprime les coups correspondants à ces équilibres
print("Les coups correspondants : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# On imprimes les scores correspondants
print("Les scores correspondants : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])


On remarque qu'il y a deux optima de Pareto pour la guerre des sexes qui sont les mêmes que pour les équilibres de Nash.

In [ ]:
# Exercice : Essayer de trouver les équilibres de Pareto (s'il existent) pour le jeu Pierre Feuille Ciseaux
rpc=[(0,0),(-1,1),(1,-1),(1,-1),(0,0),(-1,1),(-1,1),(1,-1),(0,0)]
g = Game(rpc,['R','P','C'])

## L'équilibre en stratégie dominante

Un équilibre en stratégies dominantes est toujours un équilibre de Nash mais l'inverse n'est pas vrai. Il existe deux types de stratégie dominantes : "strictement dominante" (toujours >)  et "faiblement dominante" (>=). Si on a éliminé les stratégies *strictement* dominées (EISD) et qu'il ne reste plus qu'une stratégie pour chaque joueur, alors c'est aussi le seul équilibre de Nash. Si on a éliminé les stratégies *faiblement* dominées et qu'il ne reste qu'une stratégie pour chaque joueur, alors c'est un équilibre de Nash, mais il peut y en avoir d'autres (et selon l'ordre d'élimination on tombe sur l'un ou l'autre)

In [ ]:
def getDominantStrategies(self, strict="True"):
        lignesDominees = []
        colonnesDominees = []
        findDominated = True
        while (findDominated and (len(lignesDominees) != self.size - 1) and (len(colonnesDominees) != self.size - 1)):
            findDominated = False
            #on regarde les lignes dominées
            for i in range(self.size-1) :
                ligne1 = self.scores['x'][i]
                ligne2 = self.scores['x'][i+1]
                if compare(self, ligne1, ligne2, colonnesDominees, strict):
                    if (i not in lignesDominees) : 
                        lignesDominees += [i]
                        findDominated = True
                if compare(self, ligne2, ligne1, colonnesDominees, strict):
                    if (i+1 not in lignesDominees) : 
                        lignesDominees += [i+1]
                        findDominated = True
            #on regarde les colonnes dominées
            for i in range(self.size-1) :
                c1 = self.scores['y'].transpose()[i]
                c2 = self.scores['y'].transpose()[i+1]
                if compare(self, c1, c2, lignesDominees, strict):
                    if (i not in colonnesDominees):
                        colonnesDominees += [i]
                        findDominated = True
                if compare(self, c2, c1, lignesDominees, strict):
                    if (i+1 not in colonnesDominees):
                        colonnesDominees += [i+1]
                        findDominated = True
        return result(self, lignesDominees, colonnesDominees)
    
def compare(self, l1,l2, tab, strict):
    dominated = True
    for i in range(self.size) :
        if (strict) :
            if ((l1[i] < l2[i] and i not in tab) or i in tab):
                dominated = dominated and True
            else :
                dominated = dominated and False
        else  :
            if ((l1[i] <= l2[i] and i not in tab) or i in tab):
                dominated = dominated and True
            else :
                dominated = dominated and False
    return dominated   

def result(self, lignesDominees, colonnesDominees ):
    x = list()
    y = list()
    res = list()
    
    for i in range(self.size) :
        if i not in lignesDominees : 
            x.append(i)
        if i not in colonnesDominees : 
            y.append(i)
            
    for indX in x :
        for indY in y :
            res.append((indX, indY))
            
    return res
    

    

#### Prenons le jeu de la guerre des sexes. 
On teste les stratégies dominées strictement et faiblement, si une stratégie est dominée, elle sera indiquée comme "None"

In [ ]:
gs=[(3,2),(1,1),(0,0),(2,3)]     
g = Game(gs,['Opera','Foot'])


# On récupére les indices des/de la stratégie(s) non dominée(s):
print("Avec l'élimination des stratégies strictement dominées : ")
listOfCoordinates = getDominantStrategies(g)
print("Les coordonnées des stratégies non dominées : ")
print(listOfCoordinates)

# On imprime les coups correspondants à ces équilibres
print("Les coups correspondants : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# On imprimes les scores correspondants
print("Les scores correspondants : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

print(" ")

# On récupére les indices des/de la stratégie(s) non dominée(s):
print("Avec l'élimination des stratégies faiblement dominées : ")
listOfCoordinates = getDominantStrategies(g, strict="False")
print("Les coordonnées des stratégies non dominées : ")
print(listOfCoordinates)

# On imprime les coups correspondants à ces équilibres
print("Les coups correspondants : ")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])

# On imprimes les scores correspondants
print("Les scores correspondants : ")
for cor in listOfCoordinates : 
    print(g.scores[cor[0]][cor[1]])

In [ ]:
# EXERCICE

# Simplifier le jeu abstrait suivant
exo=[(3,6),(7,1),(4,8),(5,1),(8,2),(6,1),(6,0),(6,2),(3,2)]
g = Game(exo,['A','B','C'])

# la reponse doit etre une seule case (B,B)->(8,2)
listOfCoordinates = getDominantStrategies(g)
print(listOfCoordinates)


## Intégration dans la classe Game
Toutes ces méthodes peuvent bien sûr être intégrées dans la classe Game.


### Game(tab, actions)
<ul>
<li> tab : la liste des couples de scores
<li> actions : la liste des stratégies possibles
<ul>

#### Methodes
<ul>
<li> getDominantStrategies(self, strict='True') qui imprime une liste d'indices des stratégies non dominées et qui renvoie un nouveau Game avec cette nouvelle matrice
<li> getNash(self) qui renvoie une liste d'indices des équilibres de Nash
<li> getPareto(self) qui renvoie une liste d'indices des équilibres de Pareto

<ul>

In [ ]:
%run Game.py

### Le dilemme du prisonnier

Le Dilemme du prisonnier, identifié par M. Flood and M. Dresher de la Rand Corporation en 1950, est un modèle de théorie des jeux spécialement créé pour montrer que l'équilibre de Nash n'est pas toujours une bonne idée.


#### Testons les équilibres de Nash, Pareto et les éliminations de stratégies dominées pour ce jeu.


In [ ]:
g = Game(dp,['C','D'])


listOfCoordinates = g.getNash()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print("Il existe un équilibre de Nash pour le dilemme du prisonner")
print(" ")

listOfCoordinates = g.getPareto()
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print("Il existe trois optima de Pareto pour le dilemme du prisonnier")
print(" ")

listOfCoordinates = getDominantStrategies(g)
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print("La stratégie strictement dominante pour le dilemme du prisonnier est la stratégie où les deux joueurs choisissent de trahir.")
print(" ")

listOfCoordinates = getDominantStrategies(g, strict="False")
for cor in listOfCoordinates : 
    print(g.actions[cor[0]], g.actions[cor[1]])
print("Une stratégie faiblement dominante pour le dilemme du prisonnier est la stratégie où les deux joueurs choisissent de trahir.")



## Amusons nous

### Créer une matrice de jeu au hasard

In [ ]:
x = np.random.randint(0, 5, (3,3))
y = np.random.randint(0, 5, (3,3))
couples = [(a,b) for a,b in zip(x.flatten(),y.flatten())]
g = Game(couples, None)
print(g.scores)

In [ ]:
# EXERCICE

# Trouver les équilibre de Nash, Pareto et l'élimination des stratégies dominées pour ce jeu


### Générer toutes les matrices

En Python il est facile d'énumérer tous les jeux à 2 coups à partir de valeurs fixées. Par exemple tous jeux que l'on peut construire avec les valeurs 1 et 2.
La Librairie `itertools` fournit de nombreux itérateurs efficaces, notamment pour les combinaisons, permutations et produits cartésiens. Ici c'est le produit cartésien des valeurs qui nous interesse.
On peut ensuite par exemple compter combien de jeux ont 0,1 ou plusieurs équilibres de Nash.


In [ ]:
import itertools;
import random;

def numberOfGames(valeurs, nbCoups):
    return len(valeurs)**((nbCoups**2)*2)

print("Nombre de jeux : ", numberOfGames([1,2],2))

def enumAllGames(valeurs, nbCoups):
    res = [q for q in itertools.product([p for p in itertools.product(list(valeurs), repeat=2)], repeat=nbCoups**2)]
    return [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]

n = enumAllGames([1,2],2)
print("Impression de 10 jeux aléatoires trouvés dans cet ensemble")
for i in range (10):
    print(random.choice(n))
        
def countNashEquilibria(valeurs, coups):
    results = [len(Game(i,None).getNash()) for i in enumAllGames(valeurs, coups)]
    return dict((i,results.count(i)) for i in set(results))

# Combien de jeux à 2 coups batis sur (1,2) possèdent x equilibres de Nash
countNashEquilibria([1,2],2)

# Trouver des jeux avec des contraintes particulières
Il est maintenant facile de chercher des jeux qui possèdent des contraintes particulières. Par exemple, les jeux à deux coups dont les valeurs sont prises dans (1,2) et qui ont exactement les mêmes equilibres nash et pareto

In [ ]:
nbCoups=2
res = [q for q in itertools.combinations([p for p in itertools.product([0,1,3,5], repeat=2)], nbCoups**2)]
games = [[res[j][k] for k in range(nbCoups**2)] for j in range(len(res))]
print(str(len(games))+" jeux étudiés.")
r = []
for g in games:
    if ((sorted(Game(g,None).getPareto()) == sorted(Game(g,None).getNash())) and (len(set(g)) == len(g))):
        r.append(g)
r

# Bibliographie

- John Von Neumann and Oscar Mogenstern. *Theory of Games and Economic Behavior*. Princeton Classic Editions
- Ken Binmore. *La Théorie des jeux, Une introduction.* Les éditions Arkhé
- Bernard Guerrien. *La théorie des jeux*. Economica
- Jean-Louis Boursin. *Initiation à la théorie des jeux*.  MontChretien
- [Wikipedia:Theorie des jeux](https://fr.wikipedia.org/wiki/Th%C3%A9orie_des_jeux)